In [6]:
import requests
import pandas as pd
import logging

In [17]:
gene = "OAS1"
species="human"

url = f"search/loadExpressionTissue.php?search={gene}&species={species}&type=tissue"

In [18]:
r = requests.post(url=url)
r.text

'<html><title>404: Not Found</title><body>404: Not Found</body></html>'

In [2]:
url = "https://maayanlab.cloud/archs4/gene/STAT3"
html = requests.get(url)

# Raise error if status code not "OK" Response
if html.status_code != 200:
    raise RuntimeError(
        f"Co-expression HTTP returned with error code {html.status_code}. Please try again."
    )

In [3]:
soup = BeautifulSoup(html.text, "html.parser")

In [86]:
soup.find('div', id="genedesc")

<div id="genedesc"><b>Description: </b>The protein encoded by this gene is a member of the STAT protein family. In response to cytokines and growth factors, STAT family members are phosphorylated by the receptor associated kinases, and then form homo- or heterodimers that translocate to the cell nucleus where they act as transcription activators. This protein is activated through phosphorylation in response to various cytokines and growth factors including IFNs, EGF, IL5, IL6, HGF, LIF and BMP2. This protein mediates the expression of a variety of genes in response to cell stimuli, and thus plays a key role in many cellular processes such as cell growth and apoptosis. The small GTPase Rac1 has been shown to bind and regulate the activity of this protein. PIAS3 protein is a specific inhibitor of this protein. Mutations in this gene are associated with infantile-onset multisystem autoimmune disease and hyper-immunoglobulin E syndrome. Alternative splicing results in multiple transcript v

In [76]:
soup

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Alexander Lachmann" name="author"/>
<title>ARCHS4</title>
<link href="../images/archs-icon.png?v=2" rel="icon" type="image/png"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans" rel="stylesheet"/>
<link href="https://code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css" rel="stylesheet"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet"/>
<link href="https://cdn.datatables.net/1.10.16/css/dataTables.bootstrap.min.css" rel="stylesheet"/>
<script src="https://code.jquery.com/jquery-1.12.4.js"></script>
<script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
<link href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css" rel="stylesheet"/>
<script src="https://cdn.datatables.net/1.10.16/js/jquery.dataTables.min.js"></script>
<script